In [102]:
import nltk as nltk
from nltk import sent_tokenize, word_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
import pandas as pd
import numpy as np
import math
import re
import spacy
import string
import itertools
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [103]:
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\lunar\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\lunar\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [104]:
s_stemmer = SnowballStemmer(language = 'english')

def get_stem(x):
    x = x.split()
    stem = []
    stem2 = []
    i = 0
    j = 0
    while i < len(x):
        stem.append(s_stemmer.stem(x[i]))
        i += 1
    while j < len(stem):
        if stem[j] not in stopwords.words('english'):
            stem2.append(stem[j])
        j += 1
    return " ".join(stem2)

In [105]:
def clean_sentence(x):
    if type(x) == np.float:
        return ''
    temp = x.lower()
    temp = re.sub("'", "", temp)    
    temp = re.sub("@[A-Za-z0-9_]+","", temp)
    temp = re.sub("#[A-Za-z0-9_]+","", temp)
    temp = re.sub(r'http\S+', '', temp)
    temp = re.sub('[()!?]', ' ', temp)
    temp = re.sub('\[.*?\]',' ', temp)
    temp = re.sub("[^a-z0-9]"," ", temp)
    temp = re.sub(r"[^a-z]", ' ', temp)
    temp = re.sub(r'([r][t]|[g][t])', '', temp)
    temp = re.sub(r'([ ][n][ ]|[ ][ ][ ])', '', temp)
    return temp

In [106]:
def kind_check(x):
    x = x.split()
    if 'stock' in x:
        i = 0
        while i < len(x):
            if i != len(x) - 1:
                if x[i] == 'stock':
                    if (x[i + 1] != 'price' 
                        and x[i + 1] != 'prices'
                        and x[i + 1] != 'market'
                        and x[i + 1] != 'markets'
                        and x[i - 1] != 'gamestop'
                        and x[i - 1] != 'gme'
                        and x[i - 1] != 'my'
                        and x[i - 1] != 's'
                        and ('wall','street') not in x
                        and 'shares' not in x
                        and 'share' not in x
                        and 'overvalued' not in x):
                        x[i] = 'inventory'
                        i += 1
                    else:
                        i += 1
                else:
                    i += 1
            else:
                if (x[i] == 'stock'
                        and x[i - 1] != 'gamestop'
                        and x[i - 1] != 'gme'
                        and x[i - 1] != 'my'
                        and x[i - 1] != 's'
                        and ('wall','street') not in x
                        and 'shares' not in x
                        and 'share' not in x
                        and 'overvalued' not in x):
                    x[i] = 'inventory'
                    i += 1
                else:
                    i += 1
            
    return ' '.join(x)

In [107]:
def get_words(x):
    return x.split()

In [108]:
def cosine_similarity_calc(vector_1,vector_2):

    similarity = np.dot(vector_1,vector_2)/(np.linalg.norm(vector_1)*np.linalg.norm(vector_2))

    return similarity

In [109]:
def get_similarity_value(df_in): 
    count_vec = CountVectorizer()
    dtm = count_vec.fit_transform(df_in['stem_text']).toarray()
    similarity_df = pd.DataFrame(cosine_similarity(dtm)).reset_index()
    
    return similarity_df[0]

In [110]:
st = 'kiko thinks it is a best time to buy gme stock because the stock price will continue rising in the future'

# Data 10/20 - 12/20

In [111]:
df1 = pd.read_csv('Tweets_cleaned_Oct20_Dec20.csv')

In [112]:
df1['cleaned_sentence'] = df1['cleaned_sentence'].map(clean_sentence)
df1['cleaned_sentence'] = df1['cleaned_sentence'].map(kind_check)

<ipython-input-105-93b92c955c80>:2: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if type(x) == np.float:


In [113]:
df1_1 = df1.copy()
df1_1['words'] = df1_1['cleaned_sentence'].map(get_words)
df1_2 = df1_1[['cleaned_sentence', 'words']]
df1_2['phase'] = 'Oct'

<ipython-input-113-ba48bcb2f12c>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1_2['phase'] = 'Oct'


In [114]:
df1_2['stem_text'] = df1_2['cleaned_sentence'].map(get_stem)

<ipython-input-114-b8b1e4bedca9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1_2['stem_text'] = df1_2['cleaned_sentence'].map(get_stem)


In [115]:
df1_2[:5]

,cleaned_sentence,words,phase,stem_text
0,i kinda wanna open carry now tbh i think it wo...,"[i, kinda, wanna, open, carry, now, tbh, i, th...",Oct,kinda wanna open carri tbh think would dope wa...
1,not even in inventory but what warrants gamest...,"[not, even, in, inventory, but, what, warrants...",Oct,even inventori warrant gamestop sell game new ...
2,i think fred meyer had the possibility of last...,"[i, think, fred, meyer, had, the, possibility,...",Oct,think fred meyer possibl last longer simpli be...
3,i never thought i d walk into a gamestop and t...,"[i, never, thought, i, d, walk, into, a, games...",Oct,never thought walk gamestop told inventori ps ...
4,i really feel a case could he made that next t...,"[i, really, feel, a, case, could, he, made, th...",Oct,realli feel case could made next time soni sel...


In [116]:
#df1_2.to_csv(r'C:\kiko\10_20-12_20.csv')

In [117]:
df1_3 = df1_2.copy()
df1_3.loc[-1] = ['','','Oct',get_stem(st)]
df1_3.index = df1_3.index + 1
df1_3 = df1_3.sort_index()

In [118]:
df1_3['score'] = get_similarity_value(df1_3)

In [119]:
df1_3

,cleaned_sentence,words,phase,stem_text,score
0,,,Oct,kiko think best time buy gme stock becaus stoc...,1.000000
1,i kinda wanna open carry now tbh i think it wo...,"[i, kinda, wanna, open, carry, now, tbh, i, th...",Oct,kinda wanna open carri tbh think would dope wa...,0.062622
2,not even in inventory but what warrants gamest...,"[not, even, in, inventory, but, what, warrants...",Oct,even inventori warrant gamestop sell game new ...,0.000000
3,i think fred meyer had the possibility of last...,"[i, think, fred, meyer, had, the, possibility,...",Oct,think fred meyer possibl last longer simpli be...,0.269680
4,i never thought i d walk into a gamestop and t...,"[i, never, thought, i, d, walk, into, a, games...",Oct,never thought walk gamestop told inventori ps ...,0.000000
...,...,...,...,...,...
4996,check page live up to date inventory info i th...,"[check, page, live, up, to, date, inventory, i...",Oct,check page live date inventori info think game...,0.077850
4997,i saw the contract version had likeleft but i ...,"[i, saw, the, contract, version, had, likeleft...",Oct,saw contract version likeleft feel like open l...,0.049690
4998,in australia all sold out and eb games our gam...,"[in, australia, all, sold, out, and, eb, games...",Oct,australia sold eb game gamestop wont inventori...,0.000000
4999,xbox series x was in inventory on gamestop s w...,"[xbox, series, x, was, in, inventory, on, game...",Oct,xbox seri x inventori gamestop websit less sec...,0.059235


In [120]:
df1_3['score'].mean()

0.031220965651081775

# Data 1/21 - 3/21

In [121]:
df2 = pd.read_csv('Tweets_cleaned_Jan21_Mar21.csv')
df2['cleaned_sentence'] = df2['cleaned_sentence'].map(clean_sentence)
df2['cleaned_sentence'] = df2['cleaned_sentence'].map(kind_check)

<ipython-input-105-93b92c955c80>:2: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if type(x) == np.float:


In [122]:
df2_1 = df2.copy()
df2_1['words'] = df2_1['cleaned_sentence'].map(get_words)
df2_2 = df2_1[['cleaned_sentence', 'words']]
df2_2['phase'] = 'Jan'

<ipython-input-122-2e9ccab530b0>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_2['phase'] = 'Jan'


In [123]:
df2_2['stem_text'] = df2_2['cleaned_sentence'].map(get_stem)

<ipython-input-123-409916187f92>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_2['stem_text'] = df2_2['cleaned_sentence'].map(get_stem)


In [124]:
df2_2[:5]

,cleaned_sentence,words,phase,stem_text
0,easy solution just buy him bitcoin instead of ...,"[easy, solution, just, buy, him, bitcoin, inst...",Jan,easi solut buy bitcoin instead money cohen fav...
1,cap the vision was ryan would help the sinking...,"[cap, the, vision, was, ryan, would, help, the...",Jan,cap vision ryan would help sink ship inventori...
2,because theyre never in inventory,"[because, theyre, never, in, inventory]",Jan,becaus theyr never inventori
3,stay far away from the gamestop stock madness,"[stay, far, away, from, the, gamestop, stock, ...",Jan,stay far away gamestop stock mad
4,new investors buying gamestop nyse gme stock,"[new, investors, buying, gamestop, nyse, gme, ...",Jan,new investor buy gamestop nyse gme stock


In [125]:
#df2_2.to_csv(r'C:\kiko\1_21-3_21.csv')

In [126]:
df2_3 = df2_2.copy()
df2_3.loc[-1] = ['','','Jan',get_stem(st)]
df2_3.index = df2_3.index + 1
df2_3 = df2_3.sort_index()

In [127]:
df2_3['score'] = get_similarity_value(df2_3)

In [128]:
df2_3

,cleaned_sentence,words,phase,stem_text,score
0,,,Jan,kiko think best time buy gme stock becaus stoc...,1.000000
1,easy solution just buy him bitcoin instead of ...,"[easy, solution, just, buy, him, bitcoin, inst...",Jan,easi solut buy bitcoin instead money cohen fav...,0.081650
2,cap the vision was ryan would help the sinking...,"[cap, the, vision, was, ryan, would, help, the...",Jan,cap vision ryan would help sink ship inventori...,0.055048
3,because theyre never in inventory,"[because, theyre, never, in, inventory]",Jan,becaus theyr never inventori,0.129099
4,stay far away from the gamestop stock madness,"[stay, far, away, from, the, gamestop, stock, ...",Jan,stay far away gamestop stock mad,0.210819
...,...,...,...,...,...
24986,corporate insiders at gamestop blackberry and ...,"[corporate, insiders, at, gamestop, blackberry...",Jan,corpor insid gamestop blackberri nation bevera...,0.000000
24987,gamestop robinhood inventory buying frenzy ech...,"[gamestop, robinhood, inventory, buying, frenz...",Jan,gamestop robinhood inventori buy frenzi echo t...,0.155700
24988,in cases like tesla and zoom the company can s...,"[in, cases, like, tesla, and, zoom, the, compa...",Jan,case like tesla zoom compani sell inventori hi...,0.078750
24989,so for all your talk about traders using the m...,"[so, for, all, your, talk, about, traders, usi...",Jan,talk trader use market like casino onli care r...,0.129099


In [129]:
df2_3['score'].mean()

0.12602066487725486

# Data 4/21 - 10/21

In [140]:
df3 = pd.read_csv('Tweets_cleaned_Apr21_Oct21.csv')
df3['cleaned_sentence'] = df3['cleaned_sentence'].map(clean_sentence)
df3['cleaned_sentence'] = df3['cleaned_sentence'].map(kind_check)

<ipython-input-105-93b92c955c80>:2: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if type(x) == np.float:


In [141]:
df3_1 = df3.copy()
df3_1['words'] = df3_1['cleaned_sentence'].map(get_words)
df3_2 = df3_1[['cleaned_sentence', 'words']]
df3_2['phase'] = 'Apr'

<ipython-input-141-038bdb249f56>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3_2['phase'] = 'Apr'


In [142]:
df3_2['stem_text'] = df3_2['cleaned_sentence'].map(get_stem)

<ipython-input-142-650b42a776be>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3_2['stem_text'] = df3_2['cleaned_sentence'].map(get_stem)


In [143]:
df3_2[:5]

,cleaned_sentence,words,phase,stem_text
0,breaking news get ahold of gamestop stock shar...,"[breaking, news, get, ahold, of, gamestop, sto...",Apr,break news get ahold gamestop stock share face...
1,if i have to wait one more day for those games...,"[if, i, have, to, wait, one, more, day, for, t...",Apr,wait one day gamestop stock movi come go explod
2,local politician likes gamestop gme stock soars,"[local, politician, likes, gamestop, gme, stoc...",Apr,local politician like gamestop gme stock soar
3,oddly enough besides just amazon and ebay i so...,"[oddly, enough, besides, just, amazon, and, eb...",Apr,odd enough besid amazon ebay sometim look game...
4,eevee tins back in inventory at gamestop thx,"[eevee, tins, back, in, inventory, at, gamesto...",Apr,eeve tin back inventori gamestop thx


In [144]:
df3_2.to_csv(r'C:\kiko\4_21-10_21.csv')

In [145]:
df3_3 = df3_2.copy()
df3_3.loc[-1] = ['','','Apr',get_stem(st)]
df3_3.index = df3_3.index + 1
df3_3 = df3_3.sort_index()

In [146]:
df3_3['score'] = get_similarity_value(df3_3)

In [147]:
df3_3

,cleaned_sentence,words,phase,stem_text,score
0,,,Apr,kiko think best time buy gme stock becaus stoc...,1.000000
1,breaking news get ahold of gamestop stock shar...,"[breaking, news, get, ahold, of, gamestop, sto...",Apr,break news get ahold gamestop stock share face...,0.172133
2,if i have to wait one more day for those games...,"[if, i, have, to, wait, one, more, day, for, t...",Apr,wait one day gamestop stock movi come go explod,0.172133
3,local politician likes gamestop gme stock soars,"[local, politician, likes, gamestop, gme, stoc...",Apr,local politician like gamestop gme stock soar,0.292770
4,oddly enough besides just amazon and ebay i so...,"[oddly, enough, besides, just, amazon, and, eb...",Apr,odd enough besid amazon ebay sometim look game...,0.087287
...,...,...,...,...,...
18912,etfs to turn to as fed sets stock market up fo...,"[etfs, to, turn, to, as, fed, sets, stock, mar...",Apr,etf turn fed set stock market rise inflat inte...,0.214834
18913,etfs to turn to as fed sets stock market up fo...,"[etfs, to, turn, to, as, fed, sets, stock, mar...",Apr,etf turn fed set stock market rise inflat inte...,0.214834
18914,gme was a top performing last month our shows ...,"[gme, was, a, top, performing, last, month, ou...",Apr,gme top perform last month show gamestop view ...,0.060858
18915,etfs to turn to as fed sets stock market up fo...,"[etfs, to, turn, to, as, fed, sets, stock, mar...",Apr,etf turn fed set stock market rise inflat inte...,0.214834


In [149]:
df3_3['score'].mean()

0.07780527542651411

In [150]:
print('Similarity Scores:')
print('Apr: ' + str(df3_3['score'].mean()))
print('Oct: ' + str(df1_3['score'].mean()))
print('Jan: ' + str(df2_3['score'].mean()))

Similarity Scores:
Apr: 0.07780527542651411
Oct: 0.031220965651081775
Jan: 0.12602066487725486


# Based on the cos similarity, people are likely talking about the GME stock more after the Jan phase than before the Jan phase.